# Objective: Obtain out of fold predictions on the entire training set using cross validation and then using a mean average precision IoU metric, that closely resembles the competition metric, to improve validation

In [1]:
import numpy as np
import pandas as pd

# Prepare out of fold training predictions for implementation of MAP IoU matching competition evaluation description

Load oof predictions from CNN segmentation CV kernel https://www.kaggle.com/cchadha/cnn-segmentation-cv-with-oof-preds-on-train-set/notebook

In [2]:
oof_preds0 = pd.read_csv('../input/cnn-segmentation-cv-with-oof-preds-on-train-set/submission0.csv')
#oof_preds1 = pd.read_csv('../input/cnn-segmentation-cv-with-oof-preds-on-train-set/oof_preds1.csv')
#oof_preds2 = pd.read_csv('../input/cnn-segmentation-cv-with-oof-preds-on-train-set/oof_preds2.csv')

In [3]:
oof_preds0.head()

,patientId,PredictionString
0,0fcd1318-e6ae-4d14-b466-769128285665,NaN
1,053208fa-ac79-46e2-9e78-f219ac122513,NaN
2,c1cf3255-d734-4980-bfe0-967902ad7ed9,0.5177048731732047 499 283 26 11
3,107f84d9-65b3-4a11-9c8d-921a5bed5b2f,0.6215571028343668 180 166 294 627 0.730280537...
4,c02e993c-7642-4d8a-aace-988c416ec09a,0.567282343561161 422 102 461 677 0.5501232955...


In [ ]:
#oof_preds1.head()

In [ ]:
#oof_preds2.head()

Read in training labels

In [4]:
df = pd.read_csv('../input/rsna-pneumonia-detection-challenge/stage_2_train_labels.csv')
df.head(20)

,patientId,x,y,width,height,Target
0,0004cfab-14fd-4e49-80ba-63a80b6bddd6,NaN,NaN,NaN,NaN,0
1,00313ee0-9eaa-42f4-b0ab-c148ed3241cd,NaN,NaN,NaN,NaN,0
2,00322d4d-1c29-4943-afc9-b6754be640eb,NaN,NaN,NaN,NaN,0
3,003d8fa0-6bf1-40ed-b54c-ac657f8495c5,NaN,NaN,NaN,NaN,0
4,00436515-870c-4b36-a041-de91049b9ab4,264.0,152.0,213.0,379.0,1
5,00436515-870c-4b36-a041-de91049b9ab4,562.0,152.0,256.0,453.0,1
6,00569f44-917d-4c86-a842-81832af98c30,NaN,NaN,NaN,NaN,0
7,006cec2e-6ce2-4549-bffa-eadfcd1e9970,NaN,NaN,NaN,NaN,0
8,00704310-78a8-4b38-8475-49f4573b2dbb,323.0,577.0,160.0,104.0,1
9,00704310-78a8-4b38-8475-49f4573b2dbb,695.0,575.0,162.0,137.0,1


Parse bounding box labels into correct format for Mean Average Precision IoU metric

In [5]:
df['bbox_target'] = (df['x'].astype(str) +
                    ' ' + 
                    df['y'].astype(str) +
                    ' ' +
                    df['width'].astype(str) +
                    ' ' +
                    df['height'].astype(str))
# make a bbox_target column using x, y, width and height.
df.head(10)

,patientId,x,y,width,height,Target,bbox_target
0,0004cfab-14fd-4e49-80ba-63a80b6bddd6,NaN,NaN,NaN,NaN,0,nan nan nan nan
1,00313ee0-9eaa-42f4-b0ab-c148ed3241cd,NaN,NaN,NaN,NaN,0,nan nan nan nan
2,00322d4d-1c29-4943-afc9-b6754be640eb,NaN,NaN,NaN,NaN,0,nan nan nan nan
3,003d8fa0-6bf1-40ed-b54c-ac657f8495c5,NaN,NaN,NaN,NaN,0,nan nan nan nan
4,00436515-870c-4b36-a041-de91049b9ab4,264.0,152.0,213.0,379.0,1,264.0 152.0 213.0 379.0
5,00436515-870c-4b36-a041-de91049b9ab4,562.0,152.0,256.0,453.0,1,562.0 152.0 256.0 453.0
6,00569f44-917d-4c86-a842-81832af98c30,NaN,NaN,NaN,NaN,0,nan nan nan nan
7,006cec2e-6ce2-4549-bffa-eadfcd1e9970,NaN,NaN,NaN,NaN,0,nan nan nan nan
8,00704310-78a8-4b38-8475-49f4573b2dbb,323.0,577.0,160.0,104.0,1,323.0 577.0 160.0 104.0
9,00704310-78a8-4b38-8475-49f4573b2dbb,695.0,575.0,162.0,137.0,1,695.0 575.0 162.0 137.0


In [6]:
# 공백을 기준으로 나눔, ','는 기본으로 찍힘
df.loc[:, 'bbox_target'] = df.loc[:, 'bbox_target'].map(lambda x: x.split(' '))
df.head(20)

,patientId,x,y,width,height,Target,bbox_target
0,0004cfab-14fd-4e49-80ba-63a80b6bddd6,NaN,NaN,NaN,NaN,0,"[nan, nan, nan, nan]"
1,00313ee0-9eaa-42f4-b0ab-c148ed3241cd,NaN,NaN,NaN,NaN,0,"[nan, nan, nan, nan]"
2,00322d4d-1c29-4943-afc9-b6754be640eb,NaN,NaN,NaN,NaN,0,"[nan, nan, nan, nan]"
3,003d8fa0-6bf1-40ed-b54c-ac657f8495c5,NaN,NaN,NaN,NaN,0,"[nan, nan, nan, nan]"
4,00436515-870c-4b36-a041-de91049b9ab4,264.0,152.0,213.0,379.0,1,"[264.0, 152.0, 213.0, 379.0]"
5,00436515-870c-4b36-a041-de91049b9ab4,562.0,152.0,256.0,453.0,1,"[562.0, 152.0, 256.0, 453.0]"
6,00569f44-917d-4c86-a842-81832af98c30,NaN,NaN,NaN,NaN,0,"[nan, nan, nan, nan]"
7,006cec2e-6ce2-4549-bffa-eadfcd1e9970,NaN,NaN,NaN,NaN,0,"[nan, nan, nan, nan]"
8,00704310-78a8-4b38-8475-49f4573b2dbb,323.0,577.0,160.0,104.0,1,"[323.0, 577.0, 160.0, 104.0]"
9,00704310-78a8-4b38-8475-49f4573b2dbb,695.0,575.0,162.0,137.0,1,"[695.0, 575.0, 162.0, 137.0]"


In [7]:
# 'patientId'를 기준으로 정렬 후, 동일 patientId가 있을 경우,
# bbox_target값 sum 하는데 string이라 옆으로 합쳐진다.
df = df.groupby(['patientId'], as_index = False)['bbox_target'].agg('sum')    
df.head()

,patientId,bbox_target
0,0004cfab-14fd-4e49-80ba-63a80b6bddd6,"[nan, nan, nan, nan]"
1,000924cf-0f8d-42bd-9158-1af53881a557,"[nan, nan, nan, nan]"
2,000db696-cf54-4385-b10b-6b16fbb3f985,"[316.0, 318.0, 170.0, 478.0, 660.0, 375.0, 146..."
3,000fe35a-2649-43d4-b027-e67796d412e0,"[570.0, 282.0, 269.0, 409.0, 83.0, 227.0, 296...."
4,001031d9-f904-4a23-b3e5-2c088acd19c6,"[66.0, 160.0, 373.0, 608.0, 552.0, 164.0, 376...."


Merge labels and oof preds

In [8]:
df = df.merge(oof_preds0, on = 'patientId', how = 'left')
#df = df.merge(oof_preds1, on = 'patientId', how = 'left')
#df = df.merge(oof_preds2, on = 'patientId', how = 'left')
#patientId를 기준으로 merge

In [9]:
df.head(30)

,patientId,bbox_target,PredictionString
0,0004cfab-14fd-4e49-80ba-63a80b6bddd6,"[nan, nan, nan, nan]",NaN
1,000924cf-0f8d-42bd-9158-1af53881a557,"[nan, nan, nan, nan]",NaN
2,000db696-cf54-4385-b10b-6b16fbb3f985,"[316.0, 318.0, 170.0, 478.0, 660.0, 375.0, 146...",0.5323592682701782 384 282 117 204 0.516139128...
3,000fe35a-2649-43d4-b027-e67796d412e0,"[570.0, 282.0, 269.0, 409.0, 83.0, 227.0, 296....",0.8221077651566709 488 370 356 425 0.806524258...
4,001031d9-f904-4a23-b3e5-2c088acd19c6,"[66.0, 160.0, 373.0, 608.0, 552.0, 164.0, 376....",0.757617072874964 550 538 229 280
5,0010f549-b242-4e94-87a8-57d79de215fc,"[nan, nan, nan, nan]",0.5791715817019875 243 269 180 218 0.584688553...
6,001916b8-3d30-4935-a5d1-8eaddb1646cd,"[198.0, 375.0, 114.0, 206.0]",0.7466334580538401 780 231 217 409 0.713509225...
7,0022073f-cec8-42ec-ab5f-bc2314649235,"[575.0, 232.0, 246.0, 528.0, 161.0, 230.0, 223...",0.7036652341064473 562 206 233 484 0.782359532...
8,0022995a-45eb-4cfa-9a59-cd15f5196c64,"[nan, nan, nan, nan]",0.542522326084023 423 193 167 218 0.6833132284...
9,0025d2de-bd78-4d36-9f72-e15a5e22ca82,"[nan, nan, nan, nan]",NaN


In [10]:
#3개의 파일을 merge 해서 PredictionString(x, y, default)로 뜸
df = df.fillna('') # none -> " "로 처리
df.head(30)

,patientId,bbox_target,PredictionString
0,0004cfab-14fd-4e49-80ba-63a80b6bddd6,"[nan, nan, nan, nan]",
1,000924cf-0f8d-42bd-9158-1af53881a557,"[nan, nan, nan, nan]",
2,000db696-cf54-4385-b10b-6b16fbb3f985,"[316.0, 318.0, 170.0, 478.0, 660.0, 375.0, 146...",0.5323592682701782 384 282 117 204 0.516139128...
3,000fe35a-2649-43d4-b027-e67796d412e0,"[570.0, 282.0, 269.0, 409.0, 83.0, 227.0, 296....",0.8221077651566709 488 370 356 425 0.806524258...
4,001031d9-f904-4a23-b3e5-2c088acd19c6,"[66.0, 160.0, 373.0, 608.0, 552.0, 164.0, 376....",0.757617072874964 550 538 229 280
5,0010f549-b242-4e94-87a8-57d79de215fc,"[nan, nan, nan, nan]",0.5791715817019875 243 269 180 218 0.584688553...
6,001916b8-3d30-4935-a5d1-8eaddb1646cd,"[198.0, 375.0, 114.0, 206.0]",0.7466334580538401 780 231 217 409 0.713509225...
7,0022073f-cec8-42ec-ab5f-bc2314649235,"[575.0, 232.0, 246.0, 528.0, 161.0, 230.0, 223...",0.7036652341064473 562 206 233 484 0.782359532...
8,0022995a-45eb-4cfa-9a59-cd15f5196c64,"[nan, nan, nan, nan]",0.542522326084023 423 193 167 218 0.6833132284...
9,0025d2de-bd78-4d36-9f72-e15a5e22ca82,"[nan, nan, nan, nan]",


Parse oof preds for MAP IoU

In [ ]:
#df.loc[:, 'bbox_pred'] = (df.loc[:, 'PredictionString'] +
#                         ' ' +
#                         df.loc[:, 'PredictionString_x'] +
#                         ' ' +
#                         df.loc[:, 'PredictionString_y'])
# merge한 3개 파일에서 나온 PredictionString을 bbox_pred으로 합침
#df.head()

In [11]:
#필요없는 열 제거
#df = df.drop(['PredictionString','PredictionString_x', 'PredictionString_y'], axis=1)
df.loc[:, 'bbox_pred'] = (df.loc[:, 'PredictionString'])
df = df.drop(['PredictionString'], axis=1)
df.head(20)

,patientId,bbox_target,bbox_pred
0,0004cfab-14fd-4e49-80ba-63a80b6bddd6,"[nan, nan, nan, nan]",
1,000924cf-0f8d-42bd-9158-1af53881a557,"[nan, nan, nan, nan]",
2,000db696-cf54-4385-b10b-6b16fbb3f985,"[316.0, 318.0, 170.0, 478.0, 660.0, 375.0, 146...",0.5323592682701782 384 282 117 204 0.516139128...
3,000fe35a-2649-43d4-b027-e67796d412e0,"[570.0, 282.0, 269.0, 409.0, 83.0, 227.0, 296....",0.8221077651566709 488 370 356 425 0.806524258...
4,001031d9-f904-4a23-b3e5-2c088acd19c6,"[66.0, 160.0, 373.0, 608.0, 552.0, 164.0, 376....",0.757617072874964 550 538 229 280
5,0010f549-b242-4e94-87a8-57d79de215fc,"[nan, nan, nan, nan]",0.5791715817019875 243 269 180 218 0.584688553...
6,001916b8-3d30-4935-a5d1-8eaddb1646cd,"[198.0, 375.0, 114.0, 206.0]",0.7466334580538401 780 231 217 409 0.713509225...
7,0022073f-cec8-42ec-ab5f-bc2314649235,"[575.0, 232.0, 246.0, 528.0, 161.0, 230.0, 223...",0.7036652341064473 562 206 233 484 0.782359532...
8,0022995a-45eb-4cfa-9a59-cd15f5196c64,"[nan, nan, nan, nan]",0.542522326084023 423 193 167 218 0.6833132284...
9,0025d2de-bd78-4d36-9f72-e15a5e22ca82,"[nan, nan, nan, nan]",


Stripping whitespace from PredictionString column

In [12]:
# bbox_pred 값을 문자열로 변환 후 문자열의 양끝에서 공백, 탭, 개행문자 등을 제거
df.loc[:, 'bbox_pred'] = df.loc[:, 'bbox_pred'].str.strip()
df.head(20)

,patientId,bbox_target,bbox_pred
0,0004cfab-14fd-4e49-80ba-63a80b6bddd6,"[nan, nan, nan, nan]",
1,000924cf-0f8d-42bd-9158-1af53881a557,"[nan, nan, nan, nan]",
2,000db696-cf54-4385-b10b-6b16fbb3f985,"[316.0, 318.0, 170.0, 478.0, 660.0, 375.0, 146...",0.5323592682701782 384 282 117 204 0.516139128...
3,000fe35a-2649-43d4-b027-e67796d412e0,"[570.0, 282.0, 269.0, 409.0, 83.0, 227.0, 296....",0.8221077651566709 488 370 356 425 0.806524258...
4,001031d9-f904-4a23-b3e5-2c088acd19c6,"[66.0, 160.0, 373.0, 608.0, 552.0, 164.0, 376....",0.757617072874964 550 538 229 280
5,0010f549-b242-4e94-87a8-57d79de215fc,"[nan, nan, nan, nan]",0.5791715817019875 243 269 180 218 0.584688553...
6,001916b8-3d30-4935-a5d1-8eaddb1646cd,"[198.0, 375.0, 114.0, 206.0]",0.7466334580538401 780 231 217 409 0.713509225...
7,0022073f-cec8-42ec-ab5f-bc2314649235,"[575.0, 232.0, 246.0, 528.0, 161.0, 230.0, 223...",0.7036652341064473 562 206 233 484 0.782359532...
8,0022995a-45eb-4cfa-9a59-cd15f5196c64,"[nan, nan, nan, nan]",0.542522326084023 423 193 167 218 0.6833132284...
9,0025d2de-bd78-4d36-9f72-e15a5e22ca82,"[nan, nan, nan, nan]",


In [13]:
# 공백을 기준으로 나눔, ','는 기본으로 찍힘
df.loc[:, 'bbox_pred'] = df.loc[:, 'bbox_pred'].map(lambda x: x.split(' '))
df.head(20)

,patientId,bbox_target,bbox_pred
0,0004cfab-14fd-4e49-80ba-63a80b6bddd6,"[nan, nan, nan, nan]",[]
1,000924cf-0f8d-42bd-9158-1af53881a557,"[nan, nan, nan, nan]",[]
2,000db696-cf54-4385-b10b-6b16fbb3f985,"[316.0, 318.0, 170.0, 478.0, 660.0, 375.0, 146...","[0.5323592682701782, 384, 282, 117, 204, 0.516..."
3,000fe35a-2649-43d4-b027-e67796d412e0,"[570.0, 282.0, 269.0, 409.0, 83.0, 227.0, 296....","[0.8221077651566709, 488, 370, 356, 425, 0.806..."
4,001031d9-f904-4a23-b3e5-2c088acd19c6,"[66.0, 160.0, 373.0, 608.0, 552.0, 164.0, 376....","[0.757617072874964, 550, 538, 229, 280]"
5,0010f549-b242-4e94-87a8-57d79de215fc,"[nan, nan, nan, nan]","[0.5791715817019875, 243, 269, 180, 218, 0.584..."
6,001916b8-3d30-4935-a5d1-8eaddb1646cd,"[198.0, 375.0, 114.0, 206.0]","[0.7466334580538401, 780, 231, 217, 409, 0.713..."
7,0022073f-cec8-42ec-ab5f-bc2314649235,"[575.0, 232.0, 246.0, 528.0, 161.0, 230.0, 223...","[0.7036652341064473, 562, 206, 233, 484, 0.782..."
8,0022995a-45eb-4cfa-9a59-cd15f5196c64,"[nan, nan, nan, nan]","[0.542522326084023, 423, 193, 167, 218, 0.6833..."
9,0025d2de-bd78-4d36-9f72-e15a5e22ca82,"[nan, nan, nan, nan]",[]


In [14]:
def parse_scores(x):
    if len(x)!=1:
        scores = [x[k] for k in range(0,len(x),5)]
        for score in range(len(scores)):
            scores[score] = float(scores[score])
        return np.asarray(scores)
    #score 추출

In [15]:
df.loc[:, 'bbox_scores'] = df.loc[:, 'bbox_pred'].map(parse_scores)

In [ ]:
df.head()

In [16]:
def parse_bbox(x):
    if len(x)!=1:
        bbox = [int(x[k]) for k in range(0,len(x)) if k%5 != 0]
        return np.asarray(bbox).reshape(int(len(bbox)/4),4)
    # bbox 추출

In [17]:
df.loc[:, 'bbox_preds'] = df.loc[:, 'bbox_pred'].map(parse_bbox)

In [18]:
df.head()

,patientId,bbox_target,bbox_pred,bbox_scores,bbox_preds
0,0004cfab-14fd-4e49-80ba-63a80b6bddd6,"[nan, nan, nan, nan]",[],None,None
1,000924cf-0f8d-42bd-9158-1af53881a557,"[nan, nan, nan, nan]",[],None,None
2,000db696-cf54-4385-b10b-6b16fbb3f985,"[316.0, 318.0, 170.0, 478.0, 660.0, 375.0, 146...","[0.5323592682701782, 384, 282, 117, 204, 0.516...","[0.5323592682701782, 0.5161391282115811, 0.525...","[[384, 282, 117, 204], [691, 434, 77, 205], [6..."
3,000fe35a-2649-43d4-b027-e67796d412e0,"[570.0, 282.0, 269.0, 409.0, 83.0, 227.0, 296....","[0.8221077651566709, 488, 370, 356, 425, 0.806...","[0.8221077651566709, 0.806524258692675]","[[488, 370, 356, 425], [178, 383, 256, 410]]"
4,001031d9-f904-4a23-b3e5-2c088acd19c6,"[66.0, 160.0, 373.0, 608.0, 552.0, 164.0, 376....","[0.757617072874964, 550, 538, 229, 280]",[0.757617072874964],"[[550, 538, 229, 280]]"


In [19]:
#bbox_pred 추출 후 열 제거
df = df.drop(['bbox_pred'], axis=1)

In [ ]:
df.head(20)

Edit NaN or None values to empty numpy arrays to fit MAP IoU metric implementation

In [20]:
df.loc[df['bbox_scores'].isnull(),['bbox_scores']] = df.loc[df['bbox_scores'].isnull(),'bbox_scores'].apply(lambda x: np.asarray([]))

In [ ]:
#bbox_scores null 제거 후 출력
df.head()

In [21]:
df.loc[df['bbox_preds'].isnull(),['bbox_preds']] = df.loc[df['bbox_preds'].isnull(),'bbox_preds'].apply(lambda x: np.asarray([]))

In [ ]:
#bbox_pred null 제거 후 출력
df.head()

In [22]:
def parse_target_str(x):
    if x[0] != 'nan':
        bbox = np.asarray([int(float(x[k])) for k in range(0,len(x))])
        return bbox.reshape(int(len(bbox)/4),4)

In [23]:
df.loc[:,'bbox_target'] = df.loc[:,'bbox_target'].map(parse_target_str)
df.head()
#bbox_target값 4개씩 묶어서 재 배열

,patientId,bbox_target,bbox_scores,bbox_preds
0,0004cfab-14fd-4e49-80ba-63a80b6bddd6,None,[],[]
1,000924cf-0f8d-42bd-9158-1af53881a557,None,[],[]
2,000db696-cf54-4385-b10b-6b16fbb3f985,"[[316, 318, 170, 478], [660, 375, 146, 402]]","[0.5323592682701782, 0.5161391282115811, 0.525...","[[384, 282, 117, 204], [691, 434, 77, 205], [6..."
3,000fe35a-2649-43d4-b027-e67796d412e0,"[[570, 282, 269, 409], [83, 227, 296, 438]]","[0.8221077651566709, 0.806524258692675]","[[488, 370, 356, 425], [178, 383, 256, 410]]"
4,001031d9-f904-4a23-b3e5-2c088acd19c6,"[[66, 160, 373, 608], [552, 164, 376, 676]]",[0.757617072874964],"[[550, 538, 229, 280]]"


In [24]:
df.loc[df['bbox_target'].isnull(),['bbox_target']] = df.loc[df['bbox_target'].isnull(),'bbox_target'].apply(lambda x: np.asarray([]))

In [ ]:
df.head()
# bbox_target null 제거 후 출력

# Find mean average precision IoU using implementation by chenyc15 https://www.kaggle.com/chenyc15/mean-average-precision-metric and edited herein

In [25]:
# helper function to calculate IoU
def iou(box1, box2):
    x11, y11, w1, h1 = box1
    x21, y21, w2, h2 = box2
    assert w1 * h1 > 0
    assert w2 * h2 > 0
    x12, y12 = x11 + w1, y11 + h1
    x22, y22 = x21 + w2, y21 + h2

    area1, area2 = w1 * h1, w2 * h2
    xi1, yi1, xi2, yi2 = max([x11, x21]), max([y11, y21]), min([x12, x22]), min([y12, y22])
    
    if xi2 <= xi1 or yi2 <= yi1:
        return 0
    else:
        intersect = (xi2-xi1) * (yi2-yi1)
        union = area1 + area2 - intersect
        return intersect / union

In [26]:
def map_iou(boxes_true, boxes_pred, scores, thresholds = [0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75]):
    """
    Mean average precision at differnet intersection over union (IoU) threshold
    
    input:
        boxes_true: Mx4 numpy array of ground true bounding boxes of one image. 
                    bbox format: (x1, y1, w, h)
        boxes_pred: Nx4 numpy array of predicted bounding boxes of one image. 
                    bbox format: (x1, y1, w, h)
        scores:     length N numpy array of scores associated with predicted bboxes
        thresholds: IoU shresholds to evaluate mean average precision on
    output: 
        map: mean average precision of the image
    """
    
    # According to the introduction, images with no ground truth bboxes will not be 
    # included in the map score unless there is a false positive detection (?)
        
    # return None if both are empty, don't count the image in final evaluation (?)
    if len(boxes_true) == 0 and len(boxes_pred) == 0:
        return None
    elif len(boxes_true) == 0 and len(boxes_pred) > 0:
        return 0
    elif len(boxes_true) > 0 and len(boxes_pred) == 0:
        return 0
    elif len(boxes_true) > 0 and len(boxes_pred) > 0:
        assert boxes_true.shape[1] == 4 or boxes_pred.shape[1] == 4, "boxes should be 2D arrays with shape[1]=4"
        if len(boxes_pred):
            assert len(scores) == len(boxes_pred), "boxes_pred and scores should be same length"
            # sort boxes_pred by scores in decreasing order
            boxes_pred = boxes_pred[np.argsort(scores)[::-1], :]

        map_total = 0

        # loop over thresholds
        for t in thresholds:
            matched_bt = set()
            tp, fn = 0, 0
            for i, bt in enumerate(boxes_true):
                matched = False
                for j, bp in enumerate(boxes_pred):
                    miou = iou(bt, bp)
                    if miou >= t and not matched and j not in matched_bt:
                        matched = True
                        tp += 1 # bt is matched for the first time, count as TP
                        matched_bt.add(j)
                if not matched:
                    fn += 1 # bt has no match, count as FN

            fp = len(boxes_pred) - len(matched_bt) # FP is the bp that not matched to any bt
            m = tp / (tp + fn + fp)
            map_total += m
    
    return map_total / len(thresholds)

In [ ]:
df.head(20)

In [27]:
#map_iou test, 상위 20개 행만 테스트
for row in range(20):
    print(map_iou(df['bbox_target'][row], df['bbox_preds'][row], df['bbox_scores'][row]))

None
None
0.0
0.08333333333333333
0.0
0
0.0
0.7083333333333333
0
None
0
0
0.125
None
None
None
0
0
0
None


In [28]:
#non이 아닐때만 map_iou 계산
map_scores = [
    x for x in [map_iou(df['bbox_target'][row], df['bbox_preds'][row], df['bbox_scores'][row]) for row in range(len(df))] if x is not None]

In [29]:
map_scores

[0.0,
 0.08333333333333333,
 0.0,
 0,
 0.0,
 0.7083333333333333,
 0,
 0,
 0,
 0.125,
 0,
 0,
 0,
 0,
 0,
 0,
 0.16666666666666666,
 0,
 0,
 0.1875,
 0,
 0,
 0,
 0.0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0.0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0.0,
 0,
 0,
 0.0,
 0.0,
 0.0,
 0.0,
 0,
 0.5416666666666666,
 0,
 0,
 0,
 0,
 0.0,
 0,
 0,
 0.0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0.0625,
 0,
 0,
 0,
 0.0,
 0,
 0,
 0,
 0,
 0,
 0,
 0.0,
 0,
 0,
 0,
 0,
 0.0,
 0,
 0.0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0.0,
 0.0,
 0,
 0,
 0,
 0,
 0,
 0.0,
 0,
 0,
 0,
 0.0,
 0.0,
 0,
 0.0,
 0,
 0,
 0,
 0,
 0,
 0.0,
 0,
 0,
 0.3333333333333333,
 0,
 0,
 0,
 0.0,
 0.0,
 0,
 0,
 0,
 0.16666666666666666,
 0,
 0,
 0.041666666666666664,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0.0,
 0.0,
 0.0,
 0,
 0,
 0,
 0,
 0,
 0,
 0.125,
 0.29166666666666663,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [30]:
np.mean(map_scores)

0.0026378635161321217